## SKU Creation

1. Isolate the product within the options dataframe with all their options.
2. Fill nulls using ffill method
3. Compile list of options using the optionid
4. Generate all combinations
5. Create new dataframe of options
6. Generate SKUs based on those combinations. 

## Business Problem: 
As per my conversaƟon with Allen Shimon, StrobesnMore needs a merge of their products and product 
opƟons so that every possible combinaƟon of opƟons has a unique skew. The challenge that their 
website currently faces is that their inventory lookup requires visiƟng mulƟple links and clicking into 
mulƟple dropdowns, making their current list ineffecƟve at looking up product quickly.

Example: 
Product 123 has 2 options with multiple types. 
Option 1(product feature): Magnetic VS Permanent 
Option 2(color): Blue VS Green 

CURRENTLY: Product 123 only has 1 skew (123) 
After the complete skew list is created, the following skews will be provided: 
123MBLUE 
123PBLUE 
123MGREEN 
123PGREEN 


## Data Understanding 
Two csv files were provided to demonstrate the need for this service. There is a csv with a full list of 
products and another with a list of product options. The goal will be to provide a query merging these 
two csv files so that each product has a skew associated with every combination of options associated 
with it. 
This process will require “digging into” the data and coding a query to pull the list of products and merge 
all their combination of options. To ensure quality, an understanding of the data options and their 
products will be required. 
In the product options list, a few observations have been made and need to be confirmed… 
1. It is assumed that each product can only have one unique color. 
2. The optionid is the identifier for the option categories for each product. It is assumed that each 
product can only have one unique option (for example, cannot be both magnetic and permanent) per 
optionid. 
4. The catalogid, NOT the productid in the csv appears to the proper unique identifier for each product. 
5. Client should list if there is a preffered format for SKU creation(color first, capitals, etc)





Challenges

Making this iterative process work on all products(There are 1200. Unless I want to spend 60 hours, that would take a while.)



# EDA

In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option("display.max_colwidth", 100)

In [2]:
df_prod = pd.read_csv('products.csv', encoding='unicode_escape')
len(df_prod)

1277

### Checking the products

This is a simple check of the product list, mainly to see if there are any duplicates.

In [3]:
df_prod['name'].duplicated().value_counts()

False    1254
True       23
Name: name, dtype: int64

In [4]:
df_prod[df_prod["id"].duplicated()].head(10)

,catalogid,id,name,image1,thumbnail,price,categories,description,keywords,stock
11,3486,50,Whelen 500 Series TIR6 Super-LED,assets/images/whelen-500-tir6-lighthead-blue.jpg,assets/images/thumbnails/whelen-500-tir6-black-flange-5flangeb_thumbnail.jpg,139.99,LED Lights/Body & Grille Mount,"These lightheads feature a six Super-LED panel, three Super-LEDs per color, in a single lighthea...",NaN,9930.0
12,3487,68,Whelen 500 Series Lens for LED and Halogen with Optics,assets/images/whelen-replacement-500-lens-halogen-optic.jpg,assets/images/thumbnails/whelen-replacement-500-lens-halogen-optic_thumbnail.jpg,39.99,Replacement Bulbs & Lenses/Replacement Lenses,"These are Whelen 500 Series replacement lenses. Please choose optic, which has the groves in the...",NaN,-79.0
14,3490,68,Whelen 600 Series Lens,assets/images/whelen-replacement-600-lens-halogen-optic.jpg,assets/images/thumbnails/whelen-replacement-600-lens-halogen-optic_thumbnail.jpg,76.99,Replacement Bulbs & Lenses/Replacement Lenses,These Whelen 600 Series lenses fit the below lightheads.,NaN,-97.0
17,3493,68,Whelen 700 Series Optic Lens,assets/images/whelen-replacement-700-lens-halogen-optic.jpg,assets/images/thumbnails/whelen-replacement-700-lens-halogen-optic_thumbnail.jpg,77.99,Replacement Bulbs & Lenses/Replacement Lenses,These Whelen 700 Series lenses fit the 700 Series LED lightheads.,NaN,-104.0
20,3496,68,Whelen 900 Series Optic Lens,assets/images/Whelen_900Series_OpticLens_Red.jpg,assets/images/thumbnails/Whelen_900Series_OpticLens_Red_thumbnail.jpg,77.99,Replacement Bulbs & Lenses/Replacement Lenses,These Whelen replacement lens fit the 900 Series Super-Led lightheads.,NaN,-187.0
81,3754,40,Whelen 400 Series Single Level Super-LED,assets/images/whelen-400-series-singe-level-led-amber.jpg,assets/images/thumbnails/whelen-400-series-singe-level-led-amber_thumbnail.jpg,110.99,LED Lights/Body & Grille Mount,Whelen's Super Linear 400 Series lightheads feature 5 built in scan-lcok flash patterns and are ...,NaN,33977.0
340,5401,M9,Whelen M9 Conversion Flange from M9 to 900 Series,assets/images/M9FC900.jpg,assets/images/thumbnails/m9fc900_thumbnail.jpg,41.99,LED Lights/Mounting Brackets & Accessories,NaN,NaN,-8.0
366,5431,CCSRN,Whelen CenCom Carbide Amplifier Control Module,assets/images/carbide-grp.jpg,assets/images/thumbnails/carbide-grp_thumbnail.jpg,1197.99,Sirens & Speakers/Sirens & Amplifiers,NaN,NaN,-35.0
380,5462,AV,Whelen Avenger II DUO Single Combination Linear/TIR Super-LED Dash Light,assets/images/AVENGERIIDUO.jpg,assets/images/thumbnails/avengeriiduo_thumbnail.jpg,182.99,LED Lights/Dash & Window Mount,NaN,NaN,-13.0
381,5463,AV,Whelen Avenger II TRIO Single Combination Linear/TIR Super-LED Dash Light,assets/images/AVENGERIITRIO.jpg,assets/images/thumbnails/avengeriitrio_thumbnail.jpg,194.99,LED Lights/Dash & Window Mount,NaN,NaN,-6.0


In [5]:
df_prod['name'].duplicated().sort_values(ascending=False).head(10)

638    True
491    True
511    True
582    True
607    True
774    True
917    True
918    True
920    True
921    True
Name: name, dtype: bool

In [6]:
df_prod['catalogid'].duplicated().value_counts()

False    1277
Name: catalogid, dtype: int64

In [7]:
df_prod.isnull().sum()

catalogid         0
id                1
name              1
image1          115
thumbnail       117
price             0
categories      111
description     807
keywords       1085
stock             0
dtype: int64

In [8]:
df_prod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1277 entries, 0 to 1276
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   catalogid    1277 non-null   int64  
 1   id           1276 non-null   object 
 2   name         1276 non-null   object 
 3   image1       1162 non-null   object 
 4   thumbnail    1160 non-null   object 
 5   price        1277 non-null   float64
 6   categories   1166 non-null   object 
 7   description  470 non-null    object 
 8   keywords     192 non-null    object 
 9   stock        1277 non-null   float64
dtypes: float64(2), int64(1), object(7)
memory usage: 99.9+ KB


In [9]:
df_prod['catalogid'] = df_prod['catalogid'].astype(str)

In [10]:
df_prod[df_prod['catalogid'] == '3487']['name']

12    Whelen 500 Series Lens for LED and Halogen with Optics
Name: name, dtype: object

In [71]:
df_options = pd.read_csv('options.csv',encoding='unicode_escape')
len(df_options)

4397

In [12]:
df_options.columns

Index(['optionid', 'productid', 'catalogid', 'category_id', 'featurecaption',
       'featuretype', 'featurerequired', 'sorting', 'url', 'info', 'featureid',
       'featurename', 'featureprice', 'sorting.1', 'partnumber', 'imagepath',
       'selected', 'hidden', 'optcatalogid', 'qty', 'thumbpath', 'colorcode'],
      dtype='object')

In [13]:
df_option_list = df_options.copy()

1) identify all unique products(from products dataframe)
2) identify all product options(reshape dataframe for merging with the products)
3) melt dataframe so there is a product row for every combination of options
Example:

Product 123 has 2 options with multiple types.

Option 1: Magnetic VS Permanent
Option 2: Blue VS Green

Product 123 only has 1 skew (123)

I need 4 Skews created

 - 123MBLUE - magnetic and blue
 - 123PBLUE - permanent and blue
 - 123MGREEN - magnetic and green
 - 123PGREEN - permanent and green

In [14]:
df_option_list.columns

Index(['optionid', 'productid', 'catalogid', 'category_id', 'featurecaption',
       'featuretype', 'featurerequired', 'sorting', 'url', 'info', 'featureid',
       'featurename', 'featureprice', 'sorting.1', 'partnumber', 'imagepath',
       'selected', 'hidden', 'optcatalogid', 'qty', 'thumbpath', 'colorcode'],
      dtype='object')

In [15]:
df_option_list.isnull().sum()

optionid           3544
productid          3544
catalogid          3544
category_id        3544
featurecaption     3544
featuretype        3544
featurerequired    3544
sorting            3544
url                4396
info               4395
featureid             6
featurename           6
featureprice          6
sorting.1             6
partnumber          554
imagepath          4397
selected              6
hidden                6
optcatalogid          6
qty                   6
thumbpath          4397
colorcode          2775
dtype: int64

In [16]:
df_option_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4397 entries, 0 to 4396
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   optionid         853 non-null    float64
 1   productid        853 non-null    object 
 2   catalogid        853 non-null    float64
 3   category_id      853 non-null    float64
 4   featurecaption   853 non-null    object 
 5   featuretype      853 non-null    object 
 6   featurerequired  853 non-null    float64
 7   sorting          853 non-null    float64
 8   url              1 non-null      float64
 9   info             2 non-null      object 
 10  featureid        4391 non-null   float64
 11  featurename      4391 non-null   object 
 12  featureprice     4391 non-null   float64
 13  sorting.1        4391 non-null   float64
 14  partnumber       3843 non-null   object 
 15  imagepath        0 non-null      float64
 16  selected         4391 non-null   float64
 17  hidden        

In [17]:
df_option_list.columns

Index(['optionid', 'productid', 'catalogid', 'category_id', 'featurecaption',
       'featuretype', 'featurerequired', 'sorting', 'url', 'info', 'featureid',
       'featurename', 'featureprice', 'sorting.1', 'partnumber', 'imagepath',
       'selected', 'hidden', 'optcatalogid', 'qty', 'thumbpath', 'colorcode'],
      dtype='object')

In [18]:
df_option_list = df_option_list[['optionid', 'productid', 'catalogid', 'category_id','featurecaption',
       'featuretype', 'featurerequired', 'sorting', 'url', 'info', 'featureid',
       'featurename','partnumber']]

In [19]:
df_option_list.head()

,optionid,productid,catalogid,category_id,featurecaption,featuretype,featurerequired,sorting,url,info,featureid,featurename,partnumber
0,2451.0,RB6T,3457.0,0.0,Choose your Lens Color,Dropdown,1.0,0.0,NaN,NaN,11418.0,Blue,BP
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11419.0,Clear,CP
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11420.0,Red,RP
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11421.0,Amber,AP
4,4114.0,11.1002,3474.0,0.0,Choose your Momentary Option,Dropdown,1.0,1.0,NaN,NaN,18984.0,Positive Momentary,.PM


In [20]:
df_option_list_filled = df_option_list.fillna(method='ffill')

In [21]:
df_option_list_filled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4397 entries, 0 to 4396
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   optionid         4397 non-null   float64
 1   productid        4397 non-null   object 
 2   catalogid        4397 non-null   float64
 3   category_id      4397 non-null   float64
 4   featurecaption   4397 non-null   object 
 5   featuretype      4397 non-null   object 
 6   featurerequired  4397 non-null   float64
 7   sorting          4397 non-null   float64
 8   url              416 non-null    float64
 9   info             911 non-null    object 
 10  featureid        4397 non-null   float64
 11  featurename      4397 non-null   object 
 12  partnumber       4397 non-null   object 
dtypes: float64(7), object(6)
memory usage: 446.7+ KB


# Sku format

The SKU will be comprised of the options listed with the partnumbers associated with the options.

Example:

For Product 68, LED(-1183), Non-Optic(725), Amber(-1SB) the sku will read 68-1183725-1SB

In [22]:
df_option_list_filled[df_option_list_filled['catalogid'] == 3487.0]

,optionid,productid,catalogid,category_id,featurecaption,featuretype,featurerequired,sorting,url,info,featureid,featurename,partnumber
50,2467.0,68,3487.0,0.0,Choose your Bulb,Dropdown,1.0,0.0,NaN,NaN,11487.0,Halogen,-196
51,2467.0,68,3487.0,0.0,Choose your Bulb,Dropdown,1.0,0.0,NaN,NaN,11488.0,LED,-196
52,2467.0,68,3487.0,0.0,Choose your Bulb,Dropdown,1.0,0.0,NaN,NaN,11489.0,Strobe,STROBE-
53,2468.0,68,3487.0,0.0,Choose your Lens Color,Dropdown,1.0,2.0,NaN,NaN,11490.0,Red,-50A
54,2468.0,68,3487.0,0.0,Choose your Lens Color,Dropdown,1.0,2.0,NaN,NaN,11491.0,Amber,-10A
55,2468.0,68,3487.0,0.0,Choose your Lens Color,Dropdown,1.0,2.0,NaN,NaN,11492.0,Blue,-20A
56,2468.0,68,3487.0,0.0,Choose your Lens Color,Dropdown,1.0,2.0,NaN,NaN,11493.0,Clear,-30A
57,2469.0,68,3487.0,0.0,Choose your Model,Dropdown,1.0,1.0,NaN,NaN,11494.0,Optic - Halogen,-1963237
58,2469.0,68,3487.0,0.0,Choose your Model,Dropdown,1.0,1.0,NaN,NaN,11495.0,20? Optic - LED 5MM w/o Flange,-1963583
59,2469.0,68,3487.0,0.0,Choose your Model,Dropdown,1.0,1.0,NaN,NaN,29439.0,20? Optic - TIR6 Super LED,-1963583


In [23]:
df_prod[df_prod['id'] == '68'][['id','name','catalogid']]

,id,name,catalogid
9,68,Whelen 400 Series Lens,3483
12,68,Whelen 500 Series Lens for LED and Halogen with Optics,3487
14,68,Whelen 600 Series Lens,3490
17,68,Whelen 700 Series Optic Lens,3493
20,68,Whelen 900 Series Optic Lens,3496
704,68,Whelen ION Clear Replacement Lens,6128
819,68,Whelen 700 Series Non-Optic Lens,6387


In [24]:
df_prod[df_prod['id'] == '2'][['id','name','catalogid']]

,id,name,catalogid
6,2,Whelen Warning Par36 Fairing Light,3479


In [25]:
df_option_list_filled['partnumber'].value_counts()

#NAME?                  888
R                       145
A                       139
B                       130
Equipment Info          119
C                        98
-R                       72
BB                       60
L                        58
M                        46
G                        44
J                        44
D                        40
W                        38
F                        34
RB                       33
No SAK                   30
2                        29
E                        29
V                        27
P                        24
3COLORKIT                24
K                        24
-C                       21
AA                       21
RC                       20
8SP1                     20
RR                       20
BC                       19
MBAJ94                   16
MKEZ94                   16
AC                       15
RA                       14
CUSTOM                   13
T5                       13
UNI                 

## Sample skew creation [RB6T and 11.1002]

In [26]:
df_option_list_filled.iloc[:17]

,optionid,productid,catalogid,category_id,featurecaption,featuretype,featurerequired,sorting,url,info,featureid,featurename,partnumber
0,2451.0,RB6T,3457.0,0.0,Choose your Lens Color,Dropdown,1.0,0.0,NaN,NaN,11418.0,Blue,BP
1,2451.0,RB6T,3457.0,0.0,Choose your Lens Color,Dropdown,1.0,0.0,NaN,NaN,11419.0,Clear,CP
2,2451.0,RB6T,3457.0,0.0,Choose your Lens Color,Dropdown,1.0,0.0,NaN,NaN,11420.0,Red,RP
3,2451.0,RB6T,3457.0,0.0,Choose your Lens Color,Dropdown,1.0,0.0,NaN,NaN,11421.0,Amber,AP
4,4114.0,11.1002,3474.0,0.0,Choose your Momentary Option,Dropdown,1.0,1.0,NaN,NaN,18984.0,Positive Momentary,.PM
5,4114.0,11.1002,3474.0,0.0,Choose your Momentary Option,Dropdown,1.0,1.0,NaN,NaN,18985.0,Negative/Ground Momentary,.GSM
6,2458.0,11.12,3475.0,0.0,Choose your LED Colors,Dropdown,1.0,1.0,NaN,NaN,11444.0,Blue,#NAME?
7,2458.0,11.12,3475.0,0.0,Choose your LED Colors,Dropdown,1.0,1.0,NaN,NaN,11445.0,Red/Blue,#NAME?
8,2458.0,11.12,3475.0,0.0,Choose your LED Colors,Dropdown,1.0,1.0,NaN,NaN,11446.0,Blue/White,#NAME?
9,2458.0,11.12,3475.0,0.0,Choose your LED Colors,Dropdown,1.0,1.0,NaN,NaN,11447.0,Red/White,-RC


### Product RB6T

In [27]:
df_RB6T = df_option_list_filled.iloc[:4]




### Create Skew Column

In [28]:
df_RB6T

,optionid,productid,catalogid,category_id,featurecaption,featuretype,featurerequired,sorting,url,info,featureid,featurename,partnumber
0,2451.0,RB6T,3457.0,0.0,Choose your Lens Color,Dropdown,1.0,0.0,NaN,NaN,11418.0,Blue,BP
1,2451.0,RB6T,3457.0,0.0,Choose your Lens Color,Dropdown,1.0,0.0,NaN,NaN,11419.0,Clear,CP
2,2451.0,RB6T,3457.0,0.0,Choose your Lens Color,Dropdown,1.0,0.0,NaN,NaN,11420.0,Red,RP
3,2451.0,RB6T,3457.0,0.0,Choose your Lens Color,Dropdown,1.0,0.0,NaN,NaN,11421.0,Amber,AP


In [29]:
df_RB6T['sku'] = df_RB6T['productid'] + '-' + df_RB6T['partnumber']
df_RB6T['option_description'] = df_RB6T['featurename']
df_RB6T

C:\Users\alevi\AppData\Local\Temp\ipykernel_29464\2450310250.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_RB6T['sku'] = df_RB6T['productid'] + '-' + df_RB6T['partnumber']
C:\Users\alevi\AppData\Local\Temp\ipykernel_29464\2450310250.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_RB6T['option_description'] = df_RB6T['featurename']


,optionid,productid,catalogid,category_id,featurecaption,featuretype,featurerequired,sorting,url,info,featureid,featurename,partnumber,sku,option_description
0,2451.0,RB6T,3457.0,0.0,Choose your Lens Color,Dropdown,1.0,0.0,NaN,NaN,11418.0,Blue,BP,RB6T-BP,Blue
1,2451.0,RB6T,3457.0,0.0,Choose your Lens Color,Dropdown,1.0,0.0,NaN,NaN,11419.0,Clear,CP,RB6T-CP,Clear
2,2451.0,RB6T,3457.0,0.0,Choose your Lens Color,Dropdown,1.0,0.0,NaN,NaN,11420.0,Red,RP,RB6T-RP,Red
3,2451.0,RB6T,3457.0,0.0,Choose your Lens Color,Dropdown,1.0,0.0,NaN,NaN,11421.0,Amber,AP,RB6T-AP,Amber


In [30]:
df_11 = df_option_list_filled.iloc[4:17]
df_11

,optionid,productid,catalogid,category_id,featurecaption,featuretype,featurerequired,sorting,url,info,featureid,featurename,partnumber
4,4114.0,11.1002,3474.0,0.0,Choose your Momentary Option,Dropdown,1.0,1.0,NaN,NaN,18984.0,Positive Momentary,.PM
5,4114.0,11.1002,3474.0,0.0,Choose your Momentary Option,Dropdown,1.0,1.0,NaN,NaN,18985.0,Negative/Ground Momentary,.GSM
6,2458.0,11.12,3475.0,0.0,Choose your LED Colors,Dropdown,1.0,1.0,NaN,NaN,11444.0,Blue,#NAME?
7,2458.0,11.12,3475.0,0.0,Choose your LED Colors,Dropdown,1.0,1.0,NaN,NaN,11445.0,Red/Blue,#NAME?
8,2458.0,11.12,3475.0,0.0,Choose your LED Colors,Dropdown,1.0,1.0,NaN,NaN,11446.0,Blue/White,#NAME?
9,2458.0,11.12,3475.0,0.0,Choose your LED Colors,Dropdown,1.0,1.0,NaN,NaN,11447.0,Red/White,-RC
10,2458.0,11.12,3475.0,0.0,Choose your LED Colors,Dropdown,1.0,1.0,NaN,NaN,11448.0,Amber/White,#NAME?
11,2458.0,11.12,3475.0,0.0,Choose your LED Colors,Dropdown,1.0,1.0,NaN,NaN,11449.0,Green,#NAME?
12,2458.0,11.12,3475.0,0.0,Choose your LED Colors,Dropdown,1.0,1.0,NaN,NaN,11450.0,Red,#NAME?
13,2458.0,11.12,3475.0,0.0,Choose your LED Colors,Dropdown,1.0,1.0,NaN,NaN,11451.0,Amber,#NAME?


In [31]:
df_11['partnumber'] = df_11['featurename'].apply(lambda x: 'BB' if x=='Blue'\
                                                    else 'AA' if x=='Amber'\
                                                    else 'GG' if x=='Green'\
                                                    else 'RR' if x=='Red'\
                                                    else 'AC' if x=='Amber/White'\
                                                     else 'BC' if x=='Blue/White'\
                                                     else 'RA' if x=='Red/Amber'\
                                                     else 'RB' if x=='Red/Blue'\
                                                     else 'RC' if x=='Red/White'\
                                                     else '.008' if x=='Magnetic'
                                                    else '' if x=='Permanent' else x)

C:\Users\alevi\AppData\Local\Temp\ipykernel_29464\401188226.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_11['partnumber'] = df_11['featurename'].apply(lambda x: 'BB' if x=='Blue'\


In [32]:
df_11 = df_11[df_11['productid'] == '11.12']
df_11['productid'] = '11.1200'
df_11

C:\Users\alevi\AppData\Local\Temp\ipykernel_29464\1954353845.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_11['productid'] = '11.1200'


,optionid,productid,catalogid,category_id,featurecaption,featuretype,featurerequired,sorting,url,info,featureid,featurename,partnumber
6,2458.0,11.1200,3475.0,0.0,Choose your LED Colors,Dropdown,1.0,1.0,NaN,NaN,11444.0,Blue,BB
7,2458.0,11.1200,3475.0,0.0,Choose your LED Colors,Dropdown,1.0,1.0,NaN,NaN,11445.0,Red/Blue,RB
8,2458.0,11.1200,3475.0,0.0,Choose your LED Colors,Dropdown,1.0,1.0,NaN,NaN,11446.0,Blue/White,BC
9,2458.0,11.1200,3475.0,0.0,Choose your LED Colors,Dropdown,1.0,1.0,NaN,NaN,11447.0,Red/White,RC
10,2458.0,11.1200,3475.0,0.0,Choose your LED Colors,Dropdown,1.0,1.0,NaN,NaN,11448.0,Amber/White,AC
11,2458.0,11.1200,3475.0,0.0,Choose your LED Colors,Dropdown,1.0,1.0,NaN,NaN,11449.0,Green,GG
12,2458.0,11.1200,3475.0,0.0,Choose your LED Colors,Dropdown,1.0,1.0,NaN,NaN,11450.0,Red,RR
13,2458.0,11.1200,3475.0,0.0,Choose your LED Colors,Dropdown,1.0,1.0,NaN,NaN,11451.0,Amber,AA
14,2458.0,11.1200,3475.0,0.0,Choose your LED Colors,Dropdown,1.0,1.0,NaN,NaN,11452.0,Red/Amber,RA
15,2457.0,11.1200,3475.0,0.0,Choose your Mount,Dropdown,1.0,0.0,NaN,NaN,11443.0,Magnetic,.008


In [33]:
from itertools import product

# Your three lists
options2 = list(df_11[df_11['optionid'] == 2458.0]['partnumber'])
options1 = list(df_11[df_11['optionid'] == 2457.0]['partnumber'])
# Get every permutation
permutations = list(product(options1, options2))

combos = []
# Print the permutations
for perm in permutations:
    combos.append(perm)

print(len(combos))
combos

18


[('.008', 'BB'),
 ('.008', 'RB'),
 ('.008', 'BC'),
 ('.008', 'RC'),
 ('.008', 'AC'),
 ('.008', 'GG'),
 ('.008', 'RR'),
 ('.008', 'AA'),
 ('.008', 'RA'),
 ('', 'BB'),
 ('', 'RB'),
 ('', 'BC'),
 ('', 'RC'),
 ('', 'AC'),
 ('', 'GG'),
 ('', 'RR'),
 ('', 'AA'),
 ('', 'RA')]

In [34]:
from itertools import product

# Your three lists
options2 = list(df_11[df_11['optionid'] == 2458.0]['featurename'])
options1 = list(df_11[df_11['optionid'] == 2457.0]['featurename'])
# Get every permutation
permutations = list(product(options1, options2))

combo_names = []
# Print the permutations
for perm in permutations:
    combo_names.append(perm)

print(len(combos))
combo_names

18


[('Magnetic', 'Blue'),
 ('Magnetic', 'Red/Blue'),
 ('Magnetic', 'Blue/White'),
 ('Magnetic', 'Red/White'),
 ('Magnetic', 'Amber/White'),
 ('Magnetic', 'Green'),
 ('Magnetic', 'Red'),
 ('Magnetic', 'Amber'),
 ('Magnetic', 'Red/Amber'),
 ('Permanent', 'Blue'),
 ('Permanent', 'Red/Blue'),
 ('Permanent', 'Blue/White'),
 ('Permanent', 'Red/White'),
 ('Permanent', 'Amber/White'),
 ('Permanent', 'Green'),
 ('Permanent', 'Red'),
 ('Permanent', 'Amber'),
 ('Permanent', 'Red/Amber')]

In [35]:
df_prod[df_prod['id'] == '2'][['id','name']]

,id,name
6,2,Whelen Warning Par36 Fairing Light


## Functions 

In [36]:
def join_tuple_with_underscore(t):
    return '_'.join(map(str, t))

In [37]:
sku_11_12 = pd.DataFrame(combos,columns = ['Mount','Color'])



# #Created labels for color
# skews['Color'] = skews['Color'].apply(lambda x: 'B' if x == 'Blue'\
#                                                 else 'RB' if x == 'Red/Blue'\
#                                                 else 'BW' if x =='Blue/White'\
#                                                 else 'AW' if x == 'Amber/White'\
#                                                 else 'RW' if x == 'Red/White'\
#                                                 else  'G' if x =='Green'\
#                                                 else  'R' if x == 'Red'\
#                                                 else 'A' if x =='Amber'\
#                                                 else'RA' if x == 'Red/Amber' else x)
# #Created labels for Magnetic vs Permanent
# skews['Magnetic/Permanent'] = skews['Magnetic/Permanent'].apply(lambda x: 'M' if x == 'Magnetic'\
#                                                                 else 'P' if x == 'Permanent' else x)

# #Created labels for positive vs negatice/ground momentary
# skews['Positive or Neg/Ground'] = skews['Positive or Neg/Ground'].apply(lambda x: 'P' if x == 'Positive Momentary' \
#                                                                         else 'NG' if x == 'Negative/Ground Momentary'\
#                                                                        else x)
sku_11_12['prodid'] = '11.1200'
sku_11_12 = sku_11_12[['prodid','Mount','Color']]
sku_11_12['Color'] = '-' + sku_11_12['Color']
sku_11_12['sku'] = sku_11_12.sum(axis=1)
sku_11_12['option_description'] = combo_names
sku_11_12['option_description'] = sku_11_12['option_description'].apply(join_tuple_with_underscore)

sku_11_12

,prodid,Mount,Color,sku,option_description
0,11.1200,.008,-BB,11.1200.008-BB,Magnetic_Blue
1,11.1200,.008,-RB,11.1200.008-RB,Magnetic_Red/Blue
2,11.1200,.008,-BC,11.1200.008-BC,Magnetic_Blue/White
3,11.1200,.008,-RC,11.1200.008-RC,Magnetic_Red/White
4,11.1200,.008,-AC,11.1200.008-AC,Magnetic_Amber/White
5,11.1200,.008,-GG,11.1200.008-GG,Magnetic_Green
6,11.1200,.008,-RR,11.1200.008-RR,Magnetic_Red
7,11.1200,.008,-AA,11.1200.008-AA,Magnetic_Amber
8,11.1200,.008,-RA,11.1200.008-RA,Magnetic_Red/Amber
9,11.1200,,-BB,11.1200-BB,Permanent_Blue


### More samples - products 2 and 68

Here we see that the products productid is not enough, but will be fine for the eventual sampling. The catalogid will be the unique product identifier that we will use when we finally merge the csv files. 

In [38]:
df_sample = df_option_list_filled.iloc[17:36]


In [39]:
df_prod[df_prod['id'] == '2'][['id','name','catalogid']]

,id,name,catalogid
6,2,Whelen Warning Par36 Fairing Light,3479


In [40]:
sku_2 = df_sample[df_sample['productid'] == '2'] 

In [41]:
sku_2

,optionid,productid,catalogid,category_id,featurecaption,featuretype,featurerequired,sorting,url,info,featureid,featurename,partnumber
17,2459.0,2,3479.0,0.0,Choose your LED and Lens Color,Dropdown,1.0,2.0,NaN,NaN,11453.0,Blue with Clear Lens,B00ZCR
18,2459.0,2,3479.0,0.0,Choose your LED and Lens Color,Dropdown,1.0,2.0,NaN,NaN,11454.0,White with Clear Lens,C00ZCR
19,2459.0,2,3479.0,0.0,Choose your LED and Lens Color,Dropdown,1.0,2.0,NaN,NaN,11455.0,Red with Clear Lens,R00ZCR
20,2459.0,2,3479.0,0.0,Choose your LED and Lens Color,Dropdown,1.0,2.0,NaN,NaN,11456.0,Amber with Clear Lens,A00ZCR
21,2459.0,2,3479.0,0.0,Choose your LED and Lens Color,Dropdown,1.0,2.0,NaN,NaN,18738.0,Red/Blue with Clear Lens EXTENDED LENS ONLY,J00ZCR
22,2459.0,2,3479.0,0.0,Choose your LED and Lens Color,Dropdown,1.0,2.0,NaN,NaN,18739.0,Amber with Amber Lens,A00ZAR
23,2459.0,2,3479.0,0.0,Choose your LED and Lens Color,Dropdown,1.0,2.0,NaN,NaN,18740.0,Blue with Blue Lens,B00ZBR
24,2459.0,2,3479.0,0.0,Choose your LED and Lens Color,Dropdown,1.0,2.0,NaN,NaN,18741.0,Red with Red Lens,R00ZRR
25,4858.0,2,3479.0,0.0,Choose your Lens Style,Dropdown,1.0,1.0,NaN,NaN,24256.0,Extended Lens,E
26,4858.0,2,3479.0,0.0,Choose your Lens Style,Dropdown,1.0,1.0,NaN,NaN,24257.0,Flat Lens,F


In [42]:
from itertools import product

# Your three lists
options2 = list(sku_2[sku_2['optionid'] == 2459.0]['partnumber'])
options1 = list(sku_2[sku_2['optionid'] == 4858.0]['partnumber'])
# Get every permutation
permutations = list(product(options1, options2))

combos = []
# Print the permutations
for perm in permutations:
    combos.append(perm)

print(len(combos))
combos

16


[('E', 'B00ZCR'),
 ('E', 'C00ZCR'),
 ('E', 'R00ZCR'),
 ('E', 'A00ZCR'),
 ('E', 'J00ZCR'),
 ('E', 'A00ZAR'),
 ('E', 'B00ZBR'),
 ('E', 'R00ZRR'),
 ('F', 'B00ZCR'),
 ('F', 'C00ZCR'),
 ('F', 'R00ZCR'),
 ('F', 'A00ZCR'),
 ('F', 'J00ZCR'),
 ('F', 'A00ZAR'),
 ('F', 'B00ZBR'),
 ('F', 'R00ZRR')]

In [43]:
from itertools import product

# Your three lists
options2 = list(sku_2[sku_2['optionid'] == 2459.0]['featurename'])
options1 = list(sku_2[sku_2['optionid'] == 4858.0]['featurename'])
# Get every permutation
permutations = list(product(options1, options2))

combo_names = []
# Print the permutations
for perm in permutations:
    combo_names.append(perm)

print(len(combos))
combo_names

16


[('Extended Lens', 'Blue with Clear Lens'),
 ('Extended Lens', 'White with Clear Lens'),
 ('Extended Lens', 'Red with Clear Lens'),
 ('Extended Lens', 'Amber with Clear Lens'),
 ('Extended Lens', 'Red/Blue with Clear Lens EXTENDED LENS ONLY'),
 ('Extended Lens', 'Amber with Amber Lens'),
 ('Extended Lens', 'Blue with Blue Lens'),
 ('Extended Lens', 'Red with Red Lens'),
 ('Flat Lens', 'Blue with Clear Lens'),
 ('Flat Lens', 'White with Clear Lens'),
 ('Flat Lens', 'Red with Clear Lens'),
 ('Flat Lens', 'Amber with Clear Lens'),
 ('Flat Lens', 'Red/Blue with Clear Lens EXTENDED LENS ONLY'),
 ('Flat Lens', 'Amber with Amber Lens'),
 ('Flat Lens', 'Blue with Blue Lens'),
 ('Flat Lens', 'Red with Red Lens')]

# Skus for product 2

In [44]:
skew_sample_2 = pd.DataFrame(combos,columns = ['lens_Style', 'LED_lens_color'])

skew_sample_2['prodid'] = '2'
skew_sample_2 = skew_sample_2[['prodid','lens_Style', 'LED_lens_color']]
skew_sample_2['sku'] = skew_sample_2.sum(axis=1)
skew_sample_2['option_description'] = combo_names
skew_sample_2['option_description'] = skew_sample_2['option_description'].apply(join_tuple_with_underscore)
skew_sample_2['catalogid'] = '3479'
skew_sample_2

,prodid,lens_Style,LED_lens_color,sku,option_description,catalogid
0,2,E,B00ZCR,2EB00ZCR,Extended Lens_Blue with Clear Lens,3479
1,2,E,C00ZCR,2EC00ZCR,Extended Lens_White with Clear Lens,3479
2,2,E,R00ZCR,2ER00ZCR,Extended Lens_Red with Clear Lens,3479
3,2,E,A00ZCR,2EA00ZCR,Extended Lens_Amber with Clear Lens,3479
4,2,E,J00ZCR,2EJ00ZCR,Extended Lens_Red/Blue with Clear Lens EXTENDED LENS ONLY,3479
5,2,E,A00ZAR,2EA00ZAR,Extended Lens_Amber with Amber Lens,3479
6,2,E,B00ZBR,2EB00ZBR,Extended Lens_Blue with Blue Lens,3479
7,2,E,R00ZRR,2ER00ZRR,Extended Lens_Red with Red Lens,3479
8,2,F,B00ZCR,2FB00ZCR,Flat Lens_Blue with Clear Lens,3479
9,2,F,C00ZCR,2FC00ZCR,Flat Lens_White with Clear Lens,3479


## Functions

In [45]:
df_sample[df_sample['optionid'] == 2462.0]
df_sample[df_sample['optionid'] == 2463.0]
df_sample[df_sample['optionid'] == 2464.0]

,optionid,productid,catalogid,category_id,featurecaption,featuretype,featurerequired,sorting,url,info,featureid,featurename,partnumber
34,2464.0,68,3483.0,0.0,Choose your Model,Dropdown,1.0,2.0,NaN,NaN,11473.0,Non-Optic,725
35,2464.0,68,3483.0,0.0,Choose your Model,Dropdown,1.0,2.0,NaN,NaN,11474.0,Optic,726


In [72]:
list(df_sample[df_sample['optionid'] == 2462.0]['partnumber'])

['-1183', '-3183', '-1183']

In [73]:
list(df_sample[df_sample['optionid'] == 2463.0]['partnumber'])

['-1SB', '-2SB', '-3SB', '-5SB']

In [74]:
list(df_sample[df_sample['optionid'] == 2464.0]['partnumber'])

['725', '726']

In [46]:
from itertools import product

# Your three lists
options1 = list(df_sample[df_sample['optionid'] == 2462.0]['partnumber'])
options3 = list(df_sample[df_sample['optionid'] == 2463.0]['partnumber'])
options2 = list(df_sample[df_sample['optionid'] == 2464.0]['partnumber'])
# Get every permutation
permutations = list(product(options1, options2,options3))

combos = []
# Print the permutations
for perm in permutations:
    combos.append(perm)

print(len(combos))
combos

24


[('-1183', '725', '-1SB'),
 ('-1183', '725', '-2SB'),
 ('-1183', '725', '-3SB'),
 ('-1183', '725', '-5SB'),
 ('-1183', '726', '-1SB'),
 ('-1183', '726', '-2SB'),
 ('-1183', '726', '-3SB'),
 ('-1183', '726', '-5SB'),
 ('-3183', '725', '-1SB'),
 ('-3183', '725', '-2SB'),
 ('-3183', '725', '-3SB'),
 ('-3183', '725', '-5SB'),
 ('-3183', '726', '-1SB'),
 ('-3183', '726', '-2SB'),
 ('-3183', '726', '-3SB'),
 ('-3183', '726', '-5SB'),
 ('-1183', '725', '-1SB'),
 ('-1183', '725', '-2SB'),
 ('-1183', '725', '-3SB'),
 ('-1183', '725', '-5SB'),
 ('-1183', '726', '-1SB'),
 ('-1183', '726', '-2SB'),
 ('-1183', '726', '-3SB'),
 ('-1183', '726', '-5SB')]

In [47]:
from itertools import product

# Your three lists
options1 = list(df_sample[df_sample['optionid'] == 2462.0]['featurename'])
options3 = list(df_sample[df_sample['optionid'] == 2463.0]['featurename'])
options2 = list(df_sample[df_sample['optionid'] == 2464.0]['featurename'])
# Get every permutation
permutations = list(product(options1, options2,options3))

combo_names = []
# Print the permutations
for perm in permutations:
    combo_names.append(perm)

print(len(combo_names))
combo_names

24


[('LED', 'Non-Optic', 'Amber'),
 ('LED', 'Non-Optic', 'Blue'),
 ('LED', 'Non-Optic', 'Clear'),
 ('LED', 'Non-Optic', 'Red'),
 ('LED', 'Optic', 'Amber'),
 ('LED', 'Optic', 'Blue'),
 ('LED', 'Optic', 'Clear'),
 ('LED', 'Optic', 'Red'),
 ('Strobe', 'Non-Optic', 'Amber'),
 ('Strobe', 'Non-Optic', 'Blue'),
 ('Strobe', 'Non-Optic', 'Clear'),
 ('Strobe', 'Non-Optic', 'Red'),
 ('Strobe', 'Optic', 'Amber'),
 ('Strobe', 'Optic', 'Blue'),
 ('Strobe', 'Optic', 'Clear'),
 ('Strobe', 'Optic', 'Red'),
 ('Halogen', 'Non-Optic', 'Amber'),
 ('Halogen', 'Non-Optic', 'Blue'),
 ('Halogen', 'Non-Optic', 'Clear'),
 ('Halogen', 'Non-Optic', 'Red'),
 ('Halogen', 'Optic', 'Amber'),
 ('Halogen', 'Optic', 'Blue'),
 ('Halogen', 'Optic', 'Clear'),
 ('Halogen', 'Optic', 'Red')]

### SKUs for product 68(3483)

In [48]:
skew_sample_68 = pd.DataFrame(combos,columns = ['bulb', 'model','lens_color'])

skew_sample_68['prodid'] = '68'
skew_sample_68 = skew_sample_68[['prodid','bulb', 'model','lens_color']]
skew_sample_68['sku'] = skew_sample_68.sum(axis=1)
skew_sample_68['option_description'] = combo_names
skew_sample_68['option_description'] = skew_sample_68['option_description'].apply(join_tuple_with_underscore)
skew_sample_68

,prodid,bulb,model,lens_color,sku,option_description
0,68,-1183,725,-1SB,68-1183725-1SB,LED_Non-Optic_Amber
1,68,-1183,725,-2SB,68-1183725-2SB,LED_Non-Optic_Blue
2,68,-1183,725,-3SB,68-1183725-3SB,LED_Non-Optic_Clear
3,68,-1183,725,-5SB,68-1183725-5SB,LED_Non-Optic_Red
4,68,-1183,726,-1SB,68-1183726-1SB,LED_Optic_Amber
5,68,-1183,726,-2SB,68-1183726-2SB,LED_Optic_Blue
6,68,-1183,726,-3SB,68-1183726-3SB,LED_Optic_Clear
7,68,-1183,726,-5SB,68-1183726-5SB,LED_Optic_Red
8,68,-3183,725,-1SB,68-3183725-1SB,Strobe_Non-Optic_Amber
9,68,-3183,725,-2SB,68-3183725-2SB,Strobe_Non-Optic_Blue


In [49]:
df_3487 = df_option_list.iloc[50:61]
df_3487

,optionid,productid,catalogid,category_id,featurecaption,featuretype,featurerequired,sorting,url,info,featureid,featurename,partnumber
50,2467.0,68,3487.0,0.0,Choose your Bulb,Dropdown,1.0,0.0,NaN,NaN,11487.0,Halogen,-196
51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11488.0,LED,-196
52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11489.0,Strobe,STROBE-
53,2468.0,68,3487.0,0.0,Choose your Lens Color,Dropdown,1.0,2.0,NaN,NaN,11490.0,Red,-50A
54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11491.0,Amber,-10A
55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11492.0,Blue,-20A
56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11493.0,Clear,-30A
57,2469.0,68,3487.0,0.0,Choose your Model,Dropdown,1.0,1.0,NaN,NaN,11494.0,Optic - Halogen,-1963237
58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11495.0,20? Optic - LED 5MM w/o Flange,-1963583
59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29439.0,20? Optic - TIR6 Super LED,-1963583


## For product 68(3487)

In [50]:
df_3487 = df_option_list_filled.iloc[50:61]

df_2468 = df_3487[df_3487['optionid'] == 2468.0]
df_2469 = df_3487[df_3487['optionid'] == 2469.0]

df_3487
from itertools import product

# Your three lists of the featurenames

options2 = list(df_3487[df_3487['optionid'] == 2469.0]['featurename'])
options3 = list(df_3487[df_3487['optionid'] == 2468.0]['featurename'])
# Get every permutation
permutations = list(product(options2,options3))

combo_names = []
# Print the permutations
for perm in permutations:
    combo_names.append(perm)


combo_names

# Your three lists of the partnumbers

options2 = list(df_3487[df_3487['optionid'] == 2469.0]['partnumber'])
options3 = list(df_3487[df_3487['optionid'] == 2468.0]['partnumber'])
# Get every permutation
permutations = list(product(options2,options3))

combos = []
# Print the permutations
for perm in permutations:
    combos.append(perm)
combos

combos
skew_sample_3487 = pd.DataFrame(combos,columns = ['model','lens_color'])

skew_sample_3487['prodid'] = '68'
skew_sample_3487 = skew_sample_3487[['prodid', 'model','lens_color']]
skew_sample_3487['sku'] = skew_sample_3487.sum(axis=1)
skew_sample_3487['option_description'] = combo_names
skew_sample_3487['option_description'] = skew_sample_3487['option_description'].apply(join_tuple_with_underscore)
skew_sample_3487

,prodid,model,lens_color,sku,option_description
0,68,-1963237,-50A,68-1963237-50A,Optic - Halogen_Red
1,68,-1963237,-10A,68-1963237-10A,Optic - Halogen_Amber
2,68,-1963237,-20A,68-1963237-20A,Optic - Halogen_Blue
3,68,-1963237,-30A,68-1963237-30A,Optic - Halogen_Clear
4,68,-1963583,-50A,68-1963583-50A,20? Optic - LED 5MM w/o Flange_Red
5,68,-1963583,-10A,68-1963583-10A,20? Optic - LED 5MM w/o Flange_Amber
6,68,-1963583,-20A,68-1963583-20A,20? Optic - LED 5MM w/o Flange_Blue
7,68,-1963583,-30A,68-1963583-30A,20? Optic - LED 5MM w/o Flange_Clear
8,68,-1963583,-50A,68-1963583-50A,20? Optic - TIR6 Super LED_Red
9,68,-1963583,-10A,68-1963583-10A,20? Optic - TIR6 Super LED_Amber


In [51]:
sku_11_12['catalogid'] = '3475'

In [52]:
skew_sample_68

,prodid,bulb,model,lens_color,sku,option_description
0,68,-1183,725,-1SB,68-1183725-1SB,LED_Non-Optic_Amber
1,68,-1183,725,-2SB,68-1183725-2SB,LED_Non-Optic_Blue
2,68,-1183,725,-3SB,68-1183725-3SB,LED_Non-Optic_Clear
3,68,-1183,725,-5SB,68-1183725-5SB,LED_Non-Optic_Red
4,68,-1183,726,-1SB,68-1183726-1SB,LED_Optic_Amber
5,68,-1183,726,-2SB,68-1183726-2SB,LED_Optic_Blue
6,68,-1183,726,-3SB,68-1183726-3SB,LED_Optic_Clear
7,68,-1183,726,-5SB,68-1183726-5SB,LED_Optic_Red
8,68,-3183,725,-1SB,68-3183725-1SB,Strobe_Non-Optic_Amber
9,68,-3183,725,-2SB,68-3183725-2SB,Strobe_Non-Optic_Blue


In [53]:
skew_sample_3487['catalogid'] = '3487'

In [54]:
skew_sample_68
skew_sample_68['catalogid'] = '3483'

In [55]:
skew_sample_3487
skew_sample_3487['catalogid'] = '3487'

In [56]:
sku_11_12
sku_11_12['catalogid'] = '3475'

In [57]:
df_RB6T

,optionid,productid,catalogid,category_id,featurecaption,featuretype,featurerequired,sorting,url,info,featureid,featurename,partnumber,sku,option_description
0,2451.0,RB6T,3457.0,0.0,Choose your Lens Color,Dropdown,1.0,0.0,NaN,NaN,11418.0,Blue,BP,RB6T-BP,Blue
1,2451.0,RB6T,3457.0,0.0,Choose your Lens Color,Dropdown,1.0,0.0,NaN,NaN,11419.0,Clear,CP,RB6T-CP,Clear
2,2451.0,RB6T,3457.0,0.0,Choose your Lens Color,Dropdown,1.0,0.0,NaN,NaN,11420.0,Red,RP,RB6T-RP,Red
3,2451.0,RB6T,3457.0,0.0,Choose your Lens Color,Dropdown,1.0,0.0,NaN,NaN,11421.0,Amber,AP,RB6T-AP,Amber


In [66]:
skew_sample_68 = skew_sample_68[['catalogid','sku','option_description']]
skew_sample_3487 = skew_sample_3487[['catalogid','sku','option_description']]
sku_11_12 = sku_11_12[['catalogid','sku','option_description']]
df_RB6T = df_RB6T[['catalogid','sku','option_description']]
df_RB6T['catalogid'] = '3457'
skew_sample_2 = skew_sample_2[['catalogid','sku','option_description']]
sku_sample_list = pd.concat([skew_sample_68,skew_sample_3487,sku_11_12,df_RB6T, skew_sample_2])


sku_sample_list

,catalogid,sku,option_description
0,3483,68-1183725-1SB,LED_Non-Optic_Amber
1,3483,68-1183725-2SB,LED_Non-Optic_Blue
2,3483,68-1183725-3SB,LED_Non-Optic_Clear
3,3483,68-1183725-5SB,LED_Non-Optic_Red
4,3483,68-1183726-1SB,LED_Optic_Amber
5,3483,68-1183726-2SB,LED_Optic_Blue
6,3483,68-1183726-3SB,LED_Optic_Clear
7,3483,68-1183726-5SB,LED_Optic_Red
8,3483,68-3183725-1SB,Strobe_Non-Optic_Amber
9,3483,68-3183725-2SB,Strobe_Non-Optic_Blue


In [67]:
df_prod['catalogid'] = df_prod['catalogid'].astype(str)

In [68]:
product_option_list_sample = sku_sample_list.merge(df_prod, how='inner', left_on='catalogid',right_on='catalogid')

In [69]:
product_option_list_sample.to_csv('product_option_list_sample.csv')

In [70]:
len(product_option_list_sample)

78

In [64]:
(df_RB6T)

,catalogid,sku,option_description
0,3457.0,RB6T-BP,Blue
1,3457.0,RB6T-CP,Clear
2,3457.0,RB6T-RP,Red
3,3457.0,RB6T-AP,Amber
